In [19]:
##Installing sentence-transformers library which contains vector embedding models.
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.1 MB/s eta 0:00:00


In [41]:
from sentence_transformers import SentenceTransformer
# Load the model (use 'all-MiniLM-L6-v2' for compact, fast embeddings)
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
##Testing the model for sample text
chunked_text = '''The Indian Institute of Technology Ropar is one of the eight IITs set up by the Government of India,
Ministry of Human Resource Development (MHRD) in 2008. In keeping with MoE (erstwhile) the spirit
of the IIT system, this institute is committed to provide state-of-the-art technical education in a
variety of fields, and also to facilitate transmission of knowledge using the latest developments in
pedagogy. The Indian Institute of Technology Ropar started functioning from the academic year
2008-09 from the campus of IIT Delhi, the mentor institute. The foundation stone laying ceremony
was held on 24 February 2009. The Indian Institute of Technology Ropar was registered under the
Societies' Registration Act 1860 on 29 July 2009 and later on notified under the Institutes of
Technology Act, 1961 vide gazette notification no. 59 of 1961. Now, the Institute has been declared as
an Institute of national importance. The Institute shifted to its permanent campus and currently
operates from its permanent campus. The permanent campus of IIT Ropar is spread across 501 acres
of land located in Rupnagar in the lap of nature at the banks of river Satluj. It has been awarded the 5
Star GRIHA (Green Rating for Integrated Habitat Assessment) rating, one of the highest national
ratings for Green Buildings.'''

##The embeddings are vectors of dimension 384
embeddings = model.encode(chunked_text)
print("Vector Dimension: ",embeddings.shape)


Vector Dimension:  (384,)


In [22]:
print(embeddings.shape)

(384,)


In [23]:
## Installing pdfplumber library used to extract text from the pdfs
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 42.9 MB/s eta 0:00:00


In [24]:
import pdfplumber

def extract_pdf_content(pdf_path):
    # Open the PDF
    with pdfplumber.open(pdf_path) as pdf:
        text_chunks = []
        images = []
        metadata = []

        for page_num, page in enumerate(pdf.pages):
            # Extract text
            text = page.extract_text()
            if text:
                text_chunks.append({
                    "text": text,
                    "page_number": page_num + 1  ## We are also keeping track of the page no.s
                })

    return text_chunks


In [25]:
##Creating text chunks for the UG-Handbook for the year 2021
text_chunks = extract_pdf_content("/content/IIT-Ropar-UG-Handbook-2021-15.9.21-5-3.pdf")

In [31]:
## An example chunk
text_chunks[22]

{'text': "2.4.4 Credit Limits paced programme (as defined later), then the maximum permissible number of\nregistered semesters shall be increased by two semesters.\nThe following credit limits may be adopted:\n(d) A student who is not able to complete the CGPA requirement of 5.00 for the award of\n(i) B.Tech. :19.5 Credits per semester\ndegree, after the stipulated maximum period of 6 years, can be offered to improve the\n(ii) B.Tech. with Minor : 22.5 credits per semester\ncourses / project(s) where he/she has secured a “D” grade to arrive at the CGPA of 5.00\n(iii) B.Tech. with Minor, Concentration, Honors, additional internship and 6-month\nfor the award of degree to him/her.\ninternship in 8th sem: 24 Credits per semester\n2.5.2 Conditions for Termination of Registration, Probation and Warning\n2.4.5 CGPA Requirement for Minor, Concentration, Honors, additional internship and\nIf the performance at the end of first two registered semesters is very poor, then registration\n6- Month 

In [30]:
len(text_chunks)

62

In [35]:
!pip install nltk
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [36]:
''' We are further splitting text chunks into smaller chunks as there is a limit
of 512 tokens for the sentence transformer model'''

from nltk import tokenize

def split_into_chunks(text_chunks, max_length=512):
    chunks = []
    for item in text_chunks:
        page_text = item["text"]
        page_num = item["page_number"]
        sentences = tokenize.sent_tokenize(page_text)

        current_chunk = []
        current_length = 0
        for sentence in sentences:
            if current_length + len(sentence.split()) > max_length:
                chunks.append({
                    "text": " ".join(current_chunk),
                    "page_number": page_num
                })
                current_chunk = []
                current_length = 0
            current_chunk.append(sentence)
            current_length += len(sentence.split())

        if current_chunk:  # Add remaining chunk
            chunks.append({
                "text": " ".join(current_chunk),
                "page_number": page_num
            })

    return chunks


In [37]:
chunks = split_into_chunks(text_chunks)

In [38]:
len(chunks)

108

In [40]:
chunks[22]

{'text': "Audit: A student may apply for changing a credit course to an audit one within one week\nand Dean (Academics). A student must meet his/her advisor within the first week of the new\nof the end of the mid semester examination. Audit is not allowed in any 1st year course\nsemester for the confirmation of his/her registration. The registration record should be\nand also for any core course. The credit of the courses which are audited will not be\npreserved until the semester grade card is received. counted in the final degree requirements. Various activities related to registration are listed below. The relevant dates are included in\nc. Withdrawal: A student who wants to withdraw from a course should apply within one\nthe Semester Schedule that is available before the start of the semester. week of the end of the mid semester examination. A withdrawal grade (W) will be\n2.2.2 Registration and Student Status awarded in such cases. Registration by a student confirms his/her status

In [43]:
X = model.encode(chunks[22]["text"])

In [44]:
print(X)
print(X.shape)

[-4.65272479e-02 -6.95038913e-03 -6.03071228e-03 -1.72652174e-02
  2.19187718e-02  4.41239355e-03 -8.50258861e-03 -5.03521524e-02
  1.35569805e-02  1.04894368e-02  1.16084523e-01 -2.21029557e-02
 -2.60848645e-02  7.82915391e-03 -1.10137075e-01 -4.89584468e-02
 -4.28248011e-02  1.11074895e-02  5.48803471e-02  5.77247255e-02
  2.44236402e-02 -2.41715237e-02 -4.38686684e-02  4.60683666e-02
  3.83598469e-02  2.44871564e-02  7.85222556e-03  2.85571232e-03
  7.27351382e-03 -5.89773916e-02  5.12437150e-03  9.63542797e-03
  1.98383406e-02 -6.50937064e-03  4.75707389e-02 -5.19224405e-02
  2.12981384e-02  4.56054471e-02  1.60260759e-02 -4.01435886e-03
 -6.92785084e-02 -4.86583859e-02  2.69024838e-02  1.25494171e-02
 -3.79987620e-02 -1.31329941e-02 -4.35030423e-02 -8.38266835e-02
 -4.00131047e-02  7.63970390e-02 -2.24176105e-02 -2.95484718e-02
 -9.28411782e-02 -2.19159592e-02 -7.02933744e-02 -9.32630990e-03
  1.03644110e-01 -1.24342078e-02 -5.09113632e-02  7.99295492e-03
 -1.52960867e-02  6.57831

# **Storing the embeddings in MongoDB**

In [48]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 17.8 MB/s eta 0:00:00


In [75]:
from pymongo import MongoClient
from google.colab import userdata

# Connect to DB
client = MongoClient(userdata.get('MongoURI'))
# Selecting my database
db = client.Cluster0

# Defining the JSON schema
academic_schema = {
    "bsonType": "object",
    "required": ["title", "page_number", "text", "vector"],
    "properties": {
        "title": {
            "bsonType": "string",
            "description": "must be a string and is required"
        },
        "page_number": {
            "bsonType": "int",
            "minimum": 1,
            "description": "must be an integer and at least 18"
        },
        "text": {
            "bsonType": "string",
            "description": "must be a string and is required"
        },
        "vector": {
            "bsonType": "array",
            "description": "must be an array of numbers and is required"
        }
    }
}

# Create the collection with schema validation
db.create_collection("academics", validator={"$jsonSchema": academic_schema})

Collection(Database(MongoClient(host=['cluster0-shard-00-00.d0bev.mongodb.net:27017', 'cluster0-shard-00-02.d0bev.mongodb.net:27017', 'cluster0-shard-00-01.d0bev.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-e35slv-shard-0', tls=True), 'Cluster0'), 'academics')

In [83]:
def push_vectors_to_db(chunks, title):
    for chunk in chunks:
      try:
        db.academics.insert_one({
          "title": title,
          "page_number": chunk["page_number"],
          "text": chunk["text"],
          "vector": model.encode(chunk["text"]).tolist()
        })
        print("vector inserted successfully")
      except Exception as e:
        print(f"Error inserting document: {e}")


## Pushing vectors of 2021 UG Handbook into the Database

In [84]:
push_vectors_to_db(chunks, "UG2021")

vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserted successfully
vector inserte